### Информация
- *Автор*: Киселев Кирилл
- *Дата*: вторник, 30 мая 2022 г.
- *Описание*: Блокнот для вычисления регионов для счетов-фактур

#### Requirements

In [2]:
# !pip3 install opencv-python==4.4.0.46 pandas==1.2.4

In [3]:
import cv2
import pandas as pd


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [5]:
data = pd.read_csv('/home/kirillk/datasets/OCR/ocr_schet_factura_1/test-cases-schet-factura.csv', index_col=0)

# data.loc[data['filename'] == '095.jpg', 'orientation'] = 'horizontal'

data.head()
# data.to_csv('../assets/test-cases.csv')

,filename,doc_type,doc_number,doc_date,scan_is_good,doc_type_box_left,doc_type_box_top,doc_type_box_width,doc_type_box_height,doc_number_box_left,doc_number_box_top,doc_number_box_width,doc_number_box_height,doc_date_box_left,doc_date_box_top,doc_date_box_width,doc_date_box_height
0,001.jpg,сф,6600241074,23.12.2021,True,0.046326,0.037244,0.098478,0.009955,0.169670,0.036894,0.080014,0.010479,0.271595,0.037069,0.070412,0.010654
1,002.jpg,сф,6600254174,07.02.2022,True,0.043182,0.037061,0.096808,0.009625,0.166044,0.036879,0.077140,0.009806,0.267449,0.036697,0.070754,0.010170
2,003.jpg,сф,6600013447,20.01.2022,True,0.023975,0.035199,0.096616,0.011047,0.146917,0.036286,0.078980,0.009780,0.248645,0.037191,0.071823,0.009780
3,004.jpg,сф,6015828900,13.05.2021,True,0.061055,0.059328,0.094275,0.011184,0.179674,0.059328,0.075181,0.011015,0.276813,0.059328,0.067305,0.010337
4,005.jpg,сф,6016891910,22.10.2021,True,0.042334,0.043039,0.091055,0.009589,0.156730,0.043039,0.074127,0.010511,0.251377,0.042671,0.065662,0.010326


## Исследование по вычислениям регионов

### Расположение типа документа

Отступ

In [6]:
type_indent = data['doc_type_box_left'].min()
type_indent

0.0041203696090238

Ширина

In [7]:
data['doc_type_box_left'].max()

0.0848216811504879

In [8]:
data['doc_type_box_width'].max()

0.0993167773978632

In [9]:
doc_type_region = (data['doc_type_box_left'] + data['doc_type_box_width']).max()
doc_type_region

0.1783198620922823

In [10]:
doc_type_width = (data['doc_type_box_left'] + data['doc_type_box_width']).max() - data['doc_type_box_left'].min()
doc_type_width

0.1741994924832585

In [11]:
# doc_type_region = (data['doc_type_box_left'] + data['doc_type_box_width']).max()
# doc_type_region

#### Визуализация результатов

In [14]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_schet_factura_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.2, fy=0.2)
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    y1 = int(row['doc_type_box_top'] * height)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    y2 = int((row['doc_type_box_top'] + row['doc_type_box_height']) * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image, 
                          pt1=(int(type_indent * width), 0),
                          pt2=(int(type_indent * width + doc_type_width * width), int(height)),
                          color=(0, 0, 255),
                          thickness=2)
    
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

In [11]:
list_of_deltas = []

In [12]:
%%time
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_act_1/' + row['filename'])
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    
    list_of_deltas.append(doc_type_region * width - x2)

CPU times: user 32.2 s, sys: 6.49 s, total: 38.7 s
Wall time: 34.3 s


In [13]:
# list_of_deltas

In [14]:
max(list_of_deltas)

72.08993761974737

In [15]:
min(list_of_deltas)

0.2419821034723526

In [16]:
pd.Series(list_of_deltas).describe()

count    200.000000
mean      41.615225
std       13.407599
min        0.241982
25%       34.341401
50%       44.320932
75%       50.602512
max       72.089938
dtype: float64

**Вывод:** регион для поиска типа документа имеет значения

отступ по x = 0.0042

ширина региона = 0.1742

### Расположение номера документа

In [15]:
delta_x_number_array = (data['doc_number_box_left'] - data['doc_type_box_width'] - data['doc_type_box_left'])

delta_x_number = delta_x_number_array.describe()['min']
delta_x_number

0.021826596057098205

In [16]:
delta_y_number_array = (data['doc_number_box_top'] - data['doc_type_box_height'] - data['doc_type_box_top'])

delta_y_number = delta_y_number_array.describe()['min'] 
delta_y_number

-0.012144114303614097

In [17]:
height_number = delta_y_number_array.describe()['max'] - delta_y_number_array.describe()['min'] + data['doc_number_box_height'].describe()['max']
height_number

0.01664875304025879

In [18]:
width_number = delta_x_number_array.describe()['max'] - delta_x_number_array.describe()['min'] + data['doc_number_box_width'].describe()['max']
width_number

0.08599474117943098

#### Визуализация результатов

In [21]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_schet_factura_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.7, fy=0.7)
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    y1 = int(row['doc_type_box_top'] * height)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    y2 = int((row['doc_type_box_top'] + row['doc_type_box_height']) * height)
        
    delta_x_number_ = int(delta_x_number * width)
    delta_y_number_ = int(delta_y_number * height)
    width_number_ = int(width_number * width)
    height_number_ = int(height_number * width)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image,
                            pt1=(int(x2 + delta_x_number_), 
                                int(y2 + delta_y_number_)),
                            pt2=(int(x2 + delta_x_number_ + width_number_),
                                int(y2 + delta_y_number_ + height_number_)),
                            color=(0, 0, 255),
                            thickness=2)

    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

**Вывод:** принимаем следующие параметры региона **Номер документа**

дельта по x = 0.0219

дельта по y = -0.0121

ширина региона = 0.0860

высота региона = 0.0167

 ### Расположение даты документа

In [22]:
delta_x_date_array = (data['doc_date_box_left'] - data['doc_type_box_width'] - data['doc_type_box_left'])

delta_x_date = delta_x_date_array.describe()['min']
delta_x_date

0.11578750948273657

In [23]:
delta_y_date_array = (data['doc_date_box_top'] - data['doc_type_box_height'] - data['doc_type_box_top'])

delta_y_date = delta_y_date_array.describe()['min']
delta_y_date

-0.012087579808719606

In [24]:
height_date = delta_y_date_array.describe()['max'] - delta_y_date_array.describe()['min'] + data['doc_date_box_height'].describe()['max']
height_date

0.016120370663128308

In [25]:
width_date = delta_x_date_array.describe()['max'] - delta_x_date_array.describe()['min'] + data['doc_date_box_width'].describe()['max']
width_date

0.08778993001497824

#### Визуализация результатов

In [29]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_schet_factura_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.6, fy=0.6)
    height, width, _ = image.shape
    
    x1 = int(row['doc_type_box_left'] * width)
    y1 = int(row['doc_type_box_top'] * height)
    x2 = int((row['doc_type_box_left'] + row['doc_type_box_width']) * width)
    y2 = int((row['doc_type_box_top'] + row['doc_type_box_height']) * height)
        
    delta_x_date_ = int(delta_x_date * width)
    delta_y_date_ = int(delta_y_date * height)
    width_date_ = int(width_date * width)
    height_date_ = int(height_date * width)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
    image = cv2.rectangle(img=image,
                            pt1=(int(x2 + delta_x_date_), 
                                int(y2 + delta_y_date_)),
                            pt2=(int(x2 + delta_x_date_ + width_date_),
                                int(y2 + delta_y_date_ + height_date_)),
                            color=(0, 0, 255),
                            thickness=1)

    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#     if index == 2:
#         break

**Вывод:** принимаем следующие параметры региона **Дата документа**

дельта по x = 0.1158

дельта по y = -0.0121

ширина региона = 0.0878

высота региона = 0.0162

### Дефолтные значения для номера документа

In [30]:
doc_number_x1 = data['doc_number_box_left'].min()
doc_number_x1

0.1266681366903156

In [31]:
doc_number_y1 = data['doc_number_box_top'].min()
doc_number_y1

0.0329968727363103

In [32]:
doc_number_x2 = data['doc_number_box_left'].max() + data['doc_number_box_width'].max()
doc_number_x2

0.28370910999132026

In [33]:
doc_number_y2 = data['doc_number_box_top'].max() + data['doc_number_box_height'].max()
doc_number_y2

0.08391737119194341

In [34]:
doc_number_default_width = doc_number_x2 - doc_number_x1
doc_number_default_width

0.15704097330100467

In [35]:
doc_number_default_height = doc_number_y2 - doc_number_y1
doc_number_default_height

0.05092049845563311

#### Визуализация результатов

In [45]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_schet_factura_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)
    height, width, _ = image.shape
        
    x1 = int(doc_number_x1 * width)
    y1 = int(doc_number_y1 * height)
    x2 = int(doc_number_x2 * width)
    y2 = int(doc_number_y2 * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 1)
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    if index == 2:
        break

**Вывод:** принимаем следующие параметры дефолтного региона **Номер документа**

Левый верхний угол координаиа x = 0.1267

Левый верхний угол координата y = 0.0330

ширина региона = 0.1571

высота региона = 0.0510

### Дефолтные значения для даты документа

In [38]:
doc_date_x1 = data['doc_date_box_left'].min()
doc_date_x1

0.2285029269481093

In [39]:
doc_date_y1 = data['doc_date_box_top'].min()
doc_date_y1

0.0338097087427019

In [40]:
doc_date_x2 = data['doc_date_box_left'].max() + data['doc_date_box_width'].max()
doc_date_x2

0.3743063906680121

In [41]:
doc_date_y2 = data['doc_date_box_top'].max() + data['doc_date_box_height'].max()
doc_date_y2

0.08421619899872479

In [42]:
doc_date_default_width = doc_date_x2 - doc_date_x1
doc_date_default_width

0.14580346371990277

In [43]:
doc_date_default_height = doc_date_y2 - doc_date_y1
doc_date_default_height

0.05040649025602289

#### Визуализация результатов

In [47]:
for index, row in data.iterrows():
    image = cv2.imread('/home/kirillk/datasets/OCR/ocr_schet_factura_1/' + row['filename'])
    image = cv2.resize(image, (0, 0), fx=0.6, fy=0.6)
    height, width, _ = image.shape
        
    x1 = int(doc_date_x1 * width)
    y1 = int(doc_date_y1 * height)
    x2 = int(doc_date_x2 * width)
    y2 = int(doc_date_y2 * height)
    
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 1)
    cv2.imshow('Document', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
#     if index == 2:
#         break

**Вывод:** принимаем следующие параметры дефолтного региона **Дата документа**

Левый верхний угол координаиа x = 0.2286

Левый верхний угол координата y = 0.0339

ширина региона = 0.1459

высота региона = 0.0505